In [5]:
import os
import sys

# 获取当前工作目录
current_dir = os.getcwd()
# 设置项目根目录
project_root = os.path.abspath(os.path.join(current_dir, ".."))
# 将项目根目录添加到sys.path
sys.path.insert(0, project_root)

In [6]:

from db import es
from api import deepseek
from agent import retrieval

In [ ]:
question = 'Director接收南向告警的snmp端口号是多少'

In [15]:
kws = deepseek.extract_keywords(question)
print(kws)
result = es.search_by_root_and_keywords('director', kws)['hits']['hits']
contents = [hit['_source'] for hit in result]
for content in contents:
    print(content['url'], content['keywords'], content['content'])
    print('===================================\n')

['Director', '南向告警', 'SNMP', '端口号']
性能FTP北向接口规范/topics/6ce05c0daa814722aca4ac58d84b8da8.html ['FTP性能北向接口规范', '上级网管系统', '系统管理', '修改密码', '初始密码', 'pm', 'Perf#out1!', '登录名和密码', 'TECS', 'FTP', '性能FTP北向接口规范', '内控管理', '用户名', 'Director', '配置中心'] 上级网管系统到TECS
Director取文件时，FTP用户名为pm，初始密码为Perf#out1!。可在系统管理→配置中心→内控管理页面上修改pm用户的密码。

告警SNMP北向接口规范/topics/6147725ddd864c6183501d0515dd8b3e.html ['网络管理协议', '网络事件警告', '监视网络状态', '管理工作站', 'SNMP接口简介', 'TECS Director', '接口概述', '告警SNMP北向接口规范', '第三方告警平台', '简单网络管理协议', '修改网络设备配置', '网络设备', 'SNMP', 'IETF', '互联网工程任务组', 'SNMP接口'] 简单网络管理协议（SNMP：Simple Network Management Protocol）是由互联网工程任务组（IETF：Internet
Engineering Task Force
）定义的一套网络管理协议。使用SNMP，一个管理工作站可以远程管理支持这种协议的网络设备，包括监视网络状态、修改网络设备配置、接收网络事件警告等。SNMP接口适用于TECS
Director北向对接第三方告警平台的情况。

告警SNMP北向接口规范/topics/9ef66a112134464c8b16f3ae369ea823.html ['数据链路层', '网络层', '支持的SNMP协议', 'SNMP Get', '协议标准操作', 'SNMPv3', 'SNMPv2c 基于Community 的安全模型', 'SNMPv3 基于用户的安全模型', 'DL', 'SNMP Set', '接口概述', '告警SNMP北向接口规范', 'SNMP Get Bulk', 'IETF', 'SN

In [9]:
result = retrieval.retrieve(question, 'director')
print(result)

['* TECS Director发往NMS的所有告警Trap信息中都包括两个序号标识sendNotificationId（本次发送序列号）和lastSendNotificationId（上次发送序列号）。在启用单个代理且配置一个Trap目标地址的情况下，利用这两个标识NMS可以知道是否按序无遗漏的收到来自SNMP北向的告警Trap信息。其用法为，NMS收到第一条Trap信息时，保存此Trap的sendNotificationId值，不妨记为lastId，然后收到下一条Trap时比较新Trap的lastSendNotificationId值和前面记录的lastid是否一致，若一致则说明这两条Trap是按序连续发出的，此时NMS用新Trap的sendNotificationId值更新lastId值，然后重复前述检查过程，即可保证按序无遗漏接收告警信息。若检查时发现新Trap的lastSendNotificationId值和前面记录的lastid不一致，则意味着要么这两条Trap之间存在某些Trap丢失，要么发送方发出的多条Trap在接收方发生乱序(因为SNMP协议底层采用udp协议，这两种情况都可能发生)，此时NMS可以启动容错程序，比如缓存一段时间内的Trap信息看能否重整理排序或者接收到中间丢失的Trap信息，若经容错处理后还是未能收到丢失的Trap或者不能把顺序整理成功，则NMS可以通过获取当前告警接口重新和TECS Director同步。但在启用多个代理或者单个代理配置多个Trap目标地址的情况下，这两个标识只保证是递增的。\n\n* TECS Director重启后或在发送过程中出现异常，sendNotificationId和lastSendNotificationId会自动归零，NMS可以在一段时间后通过告警接口重新和TECS Director同步。\n关于SNMP告警编号说明参见表1。\n| 名称 | 说明 | OID |\n| --- | --- | --- |\n| alarmIndex | 统一告警北向SNMP表格中告警的标识。 | .1.3.6.1.4.1.3902.4101.1.3.1.9 |\n| alarmId | 告警源产生告警时提供的告警唯一标识。 | .1.3.6.1.4.1.3902.4101.1.3.1.1 |\n| aid | 

In [20]:
a = es.search_by_content('南向告警', 'director')['hits']['hits']
contents = [hit['_source'] for hit in a]
for content in contents:
    print(content.get('url'), content['keywords'], content['content'])
    print('===================================\n')

配置指南/topics/52-配置南向参数.html ['安装与调试', '配置指南', '系统管理', '配置南向参数'] 本节介绍设置南向的资源，性能及告警通道相关参数的步骤。

对接指南/topics/配置Provider SNMP参数.html ['安装与调试', '对接指南', '对接TECS OpenStack', 'TECS OpenStack告警上报到TECS 云管系统配置说明', '配置TECS OpenStack SNMP参数'] 本任务介绍了如何配置TECS OpenStack上报告警的地址。上报告警的地址是TECS Director地址，对应TECS
Director南向网络平面地址。

对接指南/topics/1603186489658.html ['告警', 'SNMP Trap', 'QoS告警', '预定义阈值', '呈现', 'TECS Director产品SNMP南向配置', 'PaaS平台告警北向上报配置', '安装与调试', '监控', '对接PaaS的QoS告警', 'PaaS平台', 'TECS Director', '对接指南'] TECS Director产品的QoS告警是由PaaS平台进行监控。当PaaS平台根据预定义阈值触发告警时，会通过SNMP Trap将告警发送给TECS
Director进行呈现，因此需要完成PaaS平台告警北向上报配置以及TECS Director产品SNMP南向配置，从而实现上述QoS告警呈现。

端口和服务清单/topics/1622182617715.html ['kafka南向上报端口', '协议', 'TECS Director南向IP', 'MSB', '目的端口', 'EMS IP', '源端口', '客户端', '告警上报', 'TCP', '端口状态', '说明', 'Openstack管理IP', '应用端口', 'EMS', '服务端', '默认关闭', '目的IP', 'TECS Director管理IP', '源IP', '服务端/客户端', 'Openstack', '目的设备', 'TECS Director', '默认打开', 'SNMP', '端口和服务清单'] | 源设备 | 源IP | 源端口 | 目的设备 | 目的IP | 目的端口 | 协议 | 服务

## 通过llm遍历文档树去找文件

In [71]:
from utils import node_util
from api import glm

prompt_tmp = """
你是一名技术专家，我有一些运维材料文档，文档名称如下：

{names}

我现在有一个问题：{question}

你觉得它最有可能在哪些文档下？请直接输出文档名称，若有多个，请用英文逗号隔开。

注意：
1. 如果你觉得这些文档里面都没有答案，或者你不确定，都直接返回无关两个字就行
2. 只返回逗号隔开的文档名称，禁止输出任何其他文字。
输出格式如下：

{{doc1}},{{doc2}},{{doc3}}
其中大括号里面表示的是文档名称
"""

def filter(nodes):
    names = [node.name for node in nodes]
    print(f"输入: {names}")
    prompt = prompt_tmp.format(names=names, question=question)
    result = glm.chat(prompt).replace('\'','')
    print(f"输出: {result}")
    arr = [item for item in result.split(',') if item != '无关']
    last = [node for node in nodes if node.name in arr]
    return last


def process_nodes(nodes):
    result = []
    while nodes:
        filtered_nodes = filter(nodes)
        result.extend(filtered_nodes)
        all_children = []
        for node in filtered_nodes:
            all_children.extend(node.children)
        if not all_children:
            break
        nodes = all_children
    return result


In [72]:
root_path = 'D:\\Workspace\\aiops2024-challenge-dataset\\director\\'
root_nodes = node_util.parse_xml(root_path + 'nodetree.xml')
result_nodes = process_nodes(root_nodes)
# for n in result_nodes:
#     print(n.name)

输入: ['文档包简介', '描述', '安装与调试', '操作与维护', '故障管理', '参考', '性能FTP北向接口规范', '告警SNMP北向接口规范', '日志FTP北向接口规范', 'License申请操作指南', '端口和服务清单', '缩略语']
输出: 故障管理,告警SNMP北向接口规范
输入: ['故障处理', '告警与通知处理', '范围', '术语和缩略语', '接口概述', 'SNMP北向公共接口规范', 'SNMP北向告警接口规范', '参考']
输出: SNMP北向公共接口规范,SNMP北向告警接口规范
输入: ['SNMP版本支持', 'ZTE MIB根节点定义', '告警北向SNMP接口的MIB说明', '告警北向SNMP接口的功能', '告警北向SNMP接口的业务规约']
输出: 告警北向SNMP接口的MIB说明,{doc2}

由于问题涉及的是接收南向告警的SNMP端口号，这通常与接口配置和规约有关，因此我认为最相关的文档可能是描述SNMP接口MIB的文档。然而，由于提供的文档列表中没有直接提到“南向告警SNMP端口号”，我选择了与SNMP接口直接相关的文档。其他文档可能也包含相关信息，但没有足够的信息来确定它们是否包含端口号的具体细节。
输入: ['alarmIRP部分', 'csIRP部分']
输出: alarmIRP部分

输出格式：

{alarmIRP部分}


In [64]:
from pre import html_processor

# 按照url去读取文件
def read(name, url):
    dst_url = root_path + '\\documents\\' + url

    content = html_processor.html_to_markdown(dst_url)
    
    if len(content) > 1000:
        print(name)
    else:
        print(content)
        print('\n\n')
for node in result_nodes:
    read(node.name, node.url)

# 故障管理
子主题：
  * 故障处理
  * 告警与通知处理



# 告警SNMP北向接口规范
子主题：
  * 范围
  * 术语和缩略语
  * 接口概述
  * SNMP北向公共接口规范
  * SNMP北向告警接口规范
  * 参考



# SNMP北向公共接口规范
子主题：
  * SNMP版本支持
  * ZTE MIB根节点定义



# SNMP北向告警接口规范
子主题：
  * 告警北向SNMP接口的MIB说明
  * 告警北向SNMP接口的功能
  * 告警北向SNMP接口的业务规约



# 告警北向SNMP接口的MIB说明
子主题：
  * alarmIRP部分
  * csIRP部分



# 告警北向SNMP接口的业务规约
子主题：
  * 关于字符编码
  * 关于Set操作返回值说明
  * 关于SNMP Trap信息的解析
  * 关于如何保持TECS 云管系统和NMS告警数据一致
  * 通知MIB文件说明
  * 通知上报功能说明



# 关于SNMP Trap信息的解析
SNMP Trap信息的解析只能参照mib定义，使用OID去获取各变量取值。而不能忽略变量OID直接按照固定的变量顺序去取值。



# 通知MIB文件说明
通知MIB定义在zxcomc-snmp-notificationirp-
mib-v1.0.txt文件中，该MIB文件在alarmIRP部分新增notificationIRP对象，该MIB库主要定义统一北向接口中和通知属性相关的对象，功能如下：
  * ntsAttribute{ .1.3.6.1.4.1.3902.4101.10.1}分支，其下属性为通知中各字段的属性标识。
  * ntsNotifications{.1.3.6.1.4.1.3902.4101.10.2}分支，其下属性用于定义通知Trap标识，目前有通知上报Trap(ntsNotificationNew)。
  * ntsNotificationFilter{.1.3.6.1.4.1.3902.4101.10.3}分支，用于通知Trap消息的上报过滤条件，只有满足条件的通知消息才被发送到上级。



